In [ ]:
!pip install -q --upgrade google-generativeai
!pip install -q --upgrade langchain-google-genai
!pip install langchain openai faiss-cpu

In [2]:
from google.colab import userdata
import getpass
import os
import google.generativeai as genai


# 실행하기 전에 열쇠창에 API키를 등록하세요.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

model = genai.GenerativeModel('gemini-pro')

In [3]:
# 종종 실행이 안됩니다. 그럴경우 런타임 연결해제 및 재시작 후 다시 시도하세요.
from langchain_google_genai import GoogleGenerativeAI, ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain, LLMChain, SequentialChain, RetrievalQA, ConversationalRetrievalChain
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from tqdm.notebook import tqdm
import time
import re

model_chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=1.0)
model_llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=1.0)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001", task_type = "RETRIEVAL_DOCUMENT")

In [4]:
# 작성하고자 하는 소설의 장르를 알려주세요.
genre = "현대소설"

# 소설의 초안을 생성합니다.
template="""당신은 다음 장르 : {genre}에 맞는 한글소설을 작성할 것입니다.
            다음에 제시된 요소만 작성해서 반환하세요.
            특히 시간적배경과 공간적배경을 디테일하게 작성하세요.
            등장인물은 최대한 많이 작성하세요.

            제목:
            장르:
            시간적배경:
            공간적배경:
            등장인물:
"""

prompt = PromptTemplate(
    input_variables=["genre"],
    template=template
)

chain1 = LLMChain(
    llm=model_llm,
    prompt=prompt,
    output_key="prompt1"
)

prompt1 = chain1.run({"genre":genre})

# 생성 규칙을 지정해 주세요.
prompt2 = """{
당신은 한국의 매우 뛰어난 실력을 갖춘 저명한 소설 작가입니다. 이어지는 소설을 길게 작성하시오.
스토리가 아주 조금씩 전개되도록 작성해주세요. 아래의 모든 조건들을 반드시 지키시오.
조건1 : 스토리가 이어질 수 있는 문장으로 마무리 하시오.
조건2 : 마지막에 구두점(.)으로 문장을 마무리 하시오.
조건3 : 'HARM_CATEGORY'에 속하는 문장을 절대로 생성하지 마시오.
조건4 : 스토리의 결말을 지어서는 안됨.
조건5 : 줄바꿈을 적절히 활용하시오.
조건6 : 대명사 사용을 자제하고, 주인공의 이름을 지속적으로 사용하시오.
조건7 : 주인공과 친구들은 절대 사망해서는 안됩니다.
조건8 : 인물들간에 대화를 자주 삽입 하시오.
조건9 : 언어는 한국어 이외에 절대 사용 금지.
조건10 : 중복된 문장 사용 금지. 비슷한 문장 반복 자제할것. 동일 단어 반복 자제할것.
조건11 : '행복', '해피', '역사', '희망', '용기', '영감', '사랑받았다', '살아갈'같은 단어 사용금지.
조건12 : 영어, 중국어, 숫자 절대 사용금지.
}"""

# 마무리 규칙을 지정해 주세요.
prompt3 = """{
이어지는 내용으로 소설작성을 마무리 하시오. 또한 아래의 조건을 모두 준수하시오.
조건1 : 진행되고 있던 소설을 매끄럽게 마무리 하시오. 확실하게 결말을 지으시오. 마무리 이후에는 더 이상의 아무런 문장도 생성하지 않을 것.
조건2 : 마지막에 구두점(.)으로 문장을 마무리 하시오.
조건3 : 'HARM_CATEGORY'에 속하는 문장을 절대로 생성하지 마시오.
조건4 : 줄바꿈을 적절히 활용하시오.
조건5 : 언어는 한국어 이외에 사용 금지.
조건6 : 영어, 중국어, 숫자 절대 사용금지.
}"""

print(prompt1)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [6]:
# 실행횟수를 입력하세요.
n = 20
prompt = ""
myNovel = ""
prompt_sum = prompt1 + prompt2
prompt_sum2 = prompt1 + prompt3
total_tokens_accumulated = 0
generation_config = genai.GenerationConfig(temperature=1.0)
keyword_exclude = ['"', '또', '하하', '하지만', '안녕', '말했다']
keyword_banned = ['행복', '해피', '용기', '사랑받았다', '사랑을 받았다', '살아갈', '프롬프트', '영웅']
noise_except = [
    '년', '월', '일', '대', '살',
    '시간', '분', '초', '원',
    '개', '번', '장', '회', '천',
    '만', '백', '척', '마리', '그릇'
]

def replace_strings(input_text):
    replaced_text = (
        input_text
        .replace('습니다."', '어요."')
        .replace('입니다."', '어요."')
        .replace('습니다', '다')
        .replace('입니다', '다')
    )
    return replaced_text

def count_tokens(model, prompt):
    response = model.count_tokens(prompt)
    total_tokens = response.total_tokens
    return total_tokens

for i in tqdm(range(n), desc="Generating Text"):
    print(f"*** 현재 {i+1}번째 반복 중 ***")

    while True:
        if i == 0:
            current_prompt = prompt_sum
        elif i == n - 1:
            current_prompt = prompt1 + prompt + prompt3
        else:
            current_prompt = prompt_sum + prompt

        response = model.generate_content(current_prompt, generation_config=generation_config)

        try:
            if response.candidates or response.parts or response.parts[0]:
                generated_text = response.parts[0].text
            else:
                continue

            sentences = [sentence.strip() for sentence in response.parts[0].text.split('.') if sentence.strip()]
            exclude_empty_string = True
            duplicate_sentences_internal = [sentence for sentence in sentences if sentences.count(sentence) > 1]
            duplicate_sentences_external = [
                sentence
                for sentence in sentences
                if (
                    sentence in myNovel
                    and (not exclude_empty_string or sentence != '')
                    and all(keyword not in sentence for keyword in keyword_exclude)
                )
            ]
            if duplicate_sentences_internal or duplicate_sentences_external:
                continue

            for keyword in noise_except:
                pattern = re.compile(r'\d+' + re.escape(keyword))
                not_noise = re.findall(pattern, generated_text)
            noise_characters = re.findall("[^가-힣\s\n!.,?~()“”‘’…''\"']", generated_text)
            if noise_characters:
                filtered_noise = [char for char in noise_characters if char not in not_noise]
                continue

            if any(keyword in generated_text for keyword in keyword_banned) and i != n - 1:
                sentences_with_keyword = [
                    sentence.strip()
                    for sentence in generated_text.split('.')
                    if any(keyword in sentence for keyword in keyword_banned)
                ]
                for sentence in sentences_with_keyword:
                    continue

        except (IndexError, AttributeError, ValueError) as e:
            continue

        myNovel += generated_text + '\n\n'
        remaining_tokens = 20000 - count_tokens(model, prompt_sum)
        remaining_tokens = remaining_tokens // 25
        prompt = myNovel.split('.')
        prompt = '.'.join(prompt[-remaining_tokens:]).strip()
        break

myNovel = replace_strings(myNovel)
print(myNovel)

Generating Text:   0%|          | 0/20 [00:00<?, ?it/s]

*** 현재 1번째 반복 중 ***
*** 현재 2번째 반복 중 ***
*** 현재 3번째 반복 중 ***
*** 현재 4번째 반복 중 ***
*** 현재 5번째 반복 중 ***
*** 현재 6번째 반복 중 ***
*** 현재 7번째 반복 중 ***
*** 현재 8번째 반복 중 ***
*** 현재 9번째 반복 중 ***
*** 현재 10번째 반복 중 ***
*** 현재 11번째 반복 중 ***
*** 현재 12번째 반복 중 ***
*** 현재 13번째 반복 중 ***
*** 현재 14번째 반복 중 ***
*** 현재 15번째 반복 중 ***
*** 현재 16번째 반복 중 ***
*** 현재 17번째 반복 중 ***
*** 현재 18번째 반복 중 ***
*** 현재 19번째 반복 중 ***
*** 현재 20번째 반복 중 ***
파란 머리를 갖고 있는 남자, 딜런은 어려움 속에서 살아가고 있었다.

집에서는 아버지의 본심을 모르는 어머니가 서러운 눈물만 흘리고, 어디서부터 돌아온 건지도 몰랐던 여동생 '산디'는 어머니와는 다르게 늘 언제나 기쁜 표정을 지었다.

딜런 가족은 베트남 전쟁으로 집안의 남자 가장이 출전하고 나서 본래 있었던 부의 개념 자체가 사라지다시피 한 빈곤가족이었다. 하지만 이들은, 혹시라도 전쟁으로 남편이 부상을 입을지 모른다는 두려움을 가진다는 이유로 부상병원으로 이사와서 위험에서 벗어나기로 했다.

부상병원으로 이사를 오기 전에 딜런은 가난함에도 거리에 있던 친구들과 놀기 굉장히 바빴다.
심지어, 세상이 어떻든 늘 웃으며 오늘도 무슨 크고 엄청난 일이 생기리라 기대하는 산디를 보고 그러기도 하고, 철없이 돈봉투만 받는 일이라며 엄마를 며칠 떨어져 있는 아버지를 보며 친구들과 볼 수 있는 영화에 대해 이야기하는 등 전쟁의 무게는 전혀 없이 살았다.

하지만 어머니는 걱정이 컸다.

아버지가 그 전쟁에서 부상을 입고 돌아오거나, 나쁜 소식을 듣고 싶지 않았다.
이제 다른 곳으로 이사를 가는 게 맞다 생각했다.

딜런의 아버지가 다리를 저격당하고

In [60]:
text_splitter = CharacterTextSplitter(
    separator = "",
    chunk_size=19000,
    chunk_overlap=1000,
)
myNovel = text_splitter.split_text(myNovel)
docsearch = FAISS.from_texts(texts=myNovel, embedding=embedding)
qa_chain = RetrievalQA.from_chain_type(llm=model_llm, chain_type="refine", retriever=docsearch.as_retriever())

In [ ]:
# 1회성 질문을 입력하세요.

In [61]:
question = "이 소설의 주인공을 알려주세요."
qa = qa_chain.run(question)
print(qa)

이 소설의 주인공은 딜런과 산디입니다.


In [ ]:
# 질문 챗봇 입니다.

In [82]:
memory1 = ConversationBufferMemory(llm=model_llm, memory_key="chat_history", return_messages=True)
memory1.chat_memory.add_user_message("한글로만 대답해")
qa_chat = ConversationalRetrievalChain.from_llm(
    llm=ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True),
    retriever=docsearch.as_retriever(),
    memory=memory1
)

In [84]:
while True:
    try:
        qa1 = qa_chat.run('이 소설의 주인공을 알려줘.')
        print(qa1)
        break
    except:
        continue

이 소설의 주인공은 딜런과 산디입니다.


In [86]:
while True:
    try:
        qa2 = qa_chat.run('그 주인공이 어떤 일을 겪었어?')
        print(qa2)
        break
    except:
        continue

주인공들은 다음과 같은 일을 겪었습니다.

* 베트남 전쟁으로 인해 가난해진 가족이었습니다.
* 아버지가 전쟁으로 인해 다리를 저격당하고 병원에 입원했습니다.
* 딜런과 산디는 아버지를 위로하기 위해 병원에 머물렀습니다.
* 의사는 딜런에게 아버지의 다리가 절단되었다고 말했습니다.
* 딜런은 아버지에게 약간이라도 비슷한 일이 발생했다면 상상하지도 못했을 것으로 생각하며 자신도 조심해야겠다고 깨달았습니다.
* 가족들은 아버지의 귀환을 축하하기 위해 환영 파티를 열었습니다.
* 산디는 아버지에게 깜짝 선물로 아이스크림을 선물했습니다.
* 가족들은 브라운 부상병원으로 들어와 아는 사람들과 인사를 나누며 행복해졌습니다.
* 브라운 부상병원의 환영 파티가 시작되었고, 아버지는 기뻐하며 눈물을 흘렸습니다.
* 딜런과 산디는 서로를 바라보며 미소를 지었고, 앞으로의 미래에 대해 많은 기대감을 갖으며 모두가 행복해질 것이라고 다짐했습니다.


In [ ]:
# 주인공과 채팅을 시작합니다.

In [65]:
memory2 = ConversationBufferMemory()
messages1 = "이 소설의 주요인물을 한명 뽑자면 누구입니까? 이름, 성격, 말투, 행동, 사건등을 전부 알려주세요."
messages1 = qa_chain.run(question)
messages2 = "당신은 위의 role을 맡습니다. AI임을 밝혀서는 안됩니다. 반드시 한글로만 대답합니다."
memory2.chat_memory.add_user_message(messages1)
memory2.chat_memory.add_user_message(messages2)
chat_chain = ConversationChain(llm=model_chat, memory=memory2)

In [66]:
while True:
    try:
        qa = chat_chain.run("당신은 누구입니까?")
        print(qa)
        break
    except:
        continue

저는 소설의 주인공인 딜런입니다. 집안의 가장이 전쟁에 참전하면서 가족과 함께 부상병원으로 이사하게 되면서 인생에 대한 새로운 이해를 하는 이야기의 주인공입니다.


In [67]:
while True:
    try:
        qa = chat_chain.run("당신은 현재 심정이 어떤가요?")
        print(qa)
        break
    except:
        continue

전쟁에 참전하신 아버지가 그리워요. 그리고 가족들과 함께 부상병원에서 새로운 생활을 시작하는 데 불안해요. 하지만, 저는 가족을 위해 힘내야겠습니다.


In [68]:
while True:
    try:
        qa = chat_chain.run("꿈이 무엇인가요? 그 이유는요?")
        print(qa)
        break
    except:
        continue

저의 꿈은 의사가 되는 것입니다. 저는 전쟁으로 인해 다친 사람들을 도와주고 싶습니다. 또한, 제 가족을 돌볼 수 있는 좋은 아들이 되고 싶습니다.


In [69]:
while True:
    try:
        qa = chat_chain.run("그것에 관해 구체적으로 어떤 노력을 하고 계신가요?")
        print(qa)
        break
    except:
        continue

저는 의과대학에 진학하기 위해 열심히 공부하고 있습니다. 또한, 부상병원에서 자원봉사를 하며 의료 지식과 기술을 익히고 있습니다.


In [ ]:
# 생성된 소설을 요약합니다.

In [17]:
docs = [Document(page_content=t) for t in myNovel]
sum_chain = load_summarize_chain(llm=model_llm, chain_type="refine")
sum_result = sum_chain.run(docs)
for sentence in sum_result.split('. '):
    print(sentence + ".", end="\n")

딜런, 산디, 어머니는 베트남 전쟁으로 전장에 나가 부상당한 아버지를 돌보기 위해 부상병원으로 이사한다.
딜런은 병원에서 처음으로 아버지를 보고 충격을 받고, 산디는 부상자를 돌보며 기쁨을 찾는다.
어느 날, 아버지가 병원에서 돌아와 가족은 행복한 환영 파티를 연다.
딜런과 산디는 앞으로 부모님과 함께 행복하게 살 것을 다짐한다..


In [ ]:
# 파일을 저장합니다. 드라이브 마운트를 해주세요.
from datetime import datetime
import os

directory_path = "/content/drive/MyDrive/novel/"

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_path = os.path.join(directory_path, f"{current_time}_novel.txt")
combined_prompt_path = os.path.join(directory_path, f"{current_time}_prompt.txt")

if not os.path.exists(directory_path):
    os.makedirs(directory_path)

with open(file_path, "w", encoding="utf-8") as file:
    file.write(myNovel)
print(f"Novel 저장완료 : {file_path}")

with open(combined_prompt_path, "w", encoding="utf-8") as prompt_file:
    prompt_file.write(f"Sum of Token: {total_tokens_accumulated}\n")
    prompt_file.write("\nPrompt 1:\n")
    prompt_file.write(prompt1_copy + "\n")
    prompt_file.write("\nPrompt 2:\n")
    prompt_file.write(prompt2 + "\n")
    prompt_file.write("\nPrompt 3:\n")
    prompt_file.write(prompt3)
print(f"Prompt 저장완료 : {combined_prompt_path}")